## Giriş 

Bu ders şunları kapsayacaktır: 
- Fonksiyon çağrısı nedir ve kullanım alanları 
- OpenAI kullanarak fonksiyon çağrısı nasıl oluşturulur 
- Bir uygulamaya fonksiyon çağrısı nasıl entegre edilir 

## Öğrenme Hedefleri 

Bu dersi tamamladıktan sonra şunları bilecek ve anlayacaksınız: 

- Fonksiyon çağrısı kullanmanın amacı 
- OpenAI Servisi kullanarak Fonksiyon Çağrısı kurulumu 
- Uygulamanızın kullanım durumu için etkili fonksiyon çağrıları tasarlama


## Fonksiyon Çağrılarını Anlamak

Bu ders için, kullanıcıların teknik kursları bulmak için bir sohbet botu kullanmasına olanak tanıyan bir özellik oluşturmak istiyoruz. Kullanıcıların beceri seviyelerine, mevcut rollerine ve ilgi duydukları teknolojiye uygun kurslar önereceğiz.

Bunu tamamlamak için şu kombinasyonu kullanacağız:
 - Kullanıcı için bir sohbet deneyimi oluşturmak üzere `OpenAI`
 - Kullanıcının isteğine göre kurs bulmasına yardımcı olmak için `Microsoft Learn Catalog API`
 - Kullanıcının sorgusunu alıp API isteği yapmak için bir fonksiyona göndermek üzere `Fonksiyon Çağrısı`

Başlamak için, öncelikle neden fonksiyon çağrısı kullanmak istediğimize bakalım:

print("Bir sonraki istekteki Mesajlar:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # Fonksiyon yanıtını görebileceği yeni bir GPT yanıtı alın


print(second_response.choices[0].message)


### Neden Fonksiyon Çağrısı

Bu kurstaki başka bir dersi tamamladıysanız, Muazzam Dil Modelleri'nin (LLM'ler) gücünü muhtemelen anlamışsınızdır. Umarım aynı zamanda bazı sınırlamalarını da görebilirsiniz.

Fonksiyon Çağrısı, OpenAI Hizmeti'nin aşağıdaki zorlukları ele almak için tasarlanmış bir özelliğidir:

Tutarsız Yanıt Formatlama:
- Fonksiyon çağrısından önce, büyük dil modelinden gelen yanıtlar yapılandırılmamış ve tutarsızdı. Geliştiricilerin çıktının her varyasyonunu işlemek için karmaşık doğrulama kodları yazması gerekiyordu.

Dış Veri ile Sınırlı Entegrasyon:
- Bu özellikten önce, bir uygulamanın diğer bölümlerinden gelen verileri sohbet bağlamına dahil etmek zordu.

Yanıt formatlarını standartlaştırarak ve dış veri ile sorunsuz entegrasyon sağlayarak, fonksiyon çağrısı geliştirmeyi basitleştirir ve ek doğrulama mantığı ihtiyacını azaltır.

Kullanıcılar "Stockholm'deki güncel hava durumu nedir?" gibi cevaplar alamıyordu. Bunun nedeni, modellerin yalnızca verilerin eğitildiği zamana kadar sınırlı olmasıydı.

Bu sorunu gösteren aşağıdaki örneğe bakalım:

Öğrencilere doğru kursu önerebilmek için bir öğrenci veri tabanı oluşturmak istediğimizi varsayalım. Aşağıda, içerdiği veriler açısından çok benzer olan iki öğrenci açıklaması bulunmaktadır.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

Bu veriyi ayrıştırması için bir LLM'ye göndermek istiyoruz. Bu daha sonra uygulamamızda bir API'ye göndermek veya bir veritabanında saklamak için kullanılabilir.

LLM'ye hangi bilgileri istediğimizi belirttiğimiz iki özdeş istem oluşturalım:


Bunu, ürünümüz için önemli olan kısımları ayrıştırması için bir LLM'ye göndermek istiyoruz. Böylece LLM'yi yönlendirmek için iki özdeş istem oluşturabiliriz:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


Bu iki istemi oluşturduktan sonra, onları `openai.ChatCompletion` kullanarak LLM'ye göndereceğiz. İstemi `messages` değişkeninde saklıyoruz ve role olarak `user` atıyoruz. Bu, bir kullanıcının bir sohbet botuna mesaj yazmasını taklit etmek içindir.


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

Şimdi her iki isteği de LLM'ye gönderebilir ve aldığımız yanıtı inceleyebiliriz.


In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Aynı istemler ve benzer açıklamalar olmasına rağmen, `Grades` özelliğinin farklı formatlarını alabiliriz.

Yukarıdaki hücreyi birden çok kez çalıştırırsanız, format `3.7` veya `3.7 GPA` olabilir.

Bunun nedeni, LLM'nin yazılı istem biçiminde yapılandırılmamış veriler alması ve yine yapılandırılmamış veriler döndürmesidir. Bu veriyi depolarken veya kullanırken ne bekleyeceğimizi bilmek için yapılandırılmış bir formata ihtiyacımız var.

Fonksiyon çağrısı kullanarak, yapılandırılmış verileri geri aldığımızdan emin olabiliriz. Fonksiyon çağrısı kullanıldığında, LLM aslında herhangi bir fonksiyonu çağırmaz veya çalıştırmaz. Bunun yerine, LLM'nin yanıtları için takip etmesi gereken bir yapı oluştururuz. Daha sonra bu yapılandırılmış yanıtları, uygulamalarımızda hangi fonksiyonun çalıştırılacağını bilmek için kullanırız.


![Fonksiyon Çağrısı Akış Diyagramı](../../../../translated_images/tr/Function-Flow.083875364af4f4bb.webp)


Daha sonra fonksiyondan dönen değeri alabilir ve bunu LLM'ye geri gönderebiliriz. LLM, kullanıcının sorgusunu yanıtlamak için doğal dil kullanarak cevap verecektir.


### Fonksiyon çağrıları kullanmanın kullanım durumları

**Dış Araçları Çağırma**  
Chatbotlar, kullanıcılardan gelen sorulara cevap vermede iyidir. Fonksiyon çağrısı kullanarak, chatbotlar kullanıcı mesajlarını belirli görevleri tamamlamak için kullanabilir. Örneğin, bir öğrenci chatbot'a "Bu konuda daha fazla yardıma ihtiyacım olduğunu öğretmenime e-posta gönder" diyebilir. Bu, `send_email(to: string, body: string)` fonksiyonunu çağırabilir.

**API veya Veritabanı Sorguları Oluşturma**  
Kullanıcılar, doğal dil kullanarak bilgi bulabilir ve bu dil biçimlendirilmiş bir sorgu veya API isteğine dönüştürülebilir. Bunun bir örneği, "Son ödevi tamamlayan öğrenciler kimler" diye soran bir öğretmen olabilir; bu, `get_completed(student_name: string, assignment: int, current_status: string)` adlı bir fonksiyonu çağırabilir.

**Yapılandırılmış Veri Oluşturma**  
Kullanıcılar bir metin bloğunu veya CSV'yi alıp önemli bilgileri çıkarmak için LLM'yi kullanabilir. Örneğin, bir öğrenci barış anlaşmaları hakkında bir Wikipedia makalesini AI flash kartları oluşturmak için dönüştürebilir. Bu, `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)` adlı bir fonksiyon kullanılarak yapılabilir.


## 2. İlk Fonksiyon Çağrınızı Oluşturma

Bir fonksiyon çağrısı oluşturma süreci 3 ana adımdan oluşur:  
1. Fonksiyonlarınızın bir listesi ve bir kullanıcı mesajı ile Chat Completions API'sini çağırmak  
2. Bir işlem gerçekleştirmek için modelin yanıtını okumak, yani bir fonksiyonu veya API Çağrısını yürütmek  
3. Fonksiyonunuzdan gelen yanıtla Chat Completions API'sine başka bir çağrı yapmak ve bu bilgiyi kullanıcıya yanıt oluşturmak için kullanmak.


![Bir Fonksiyon Çağrısının Akışı](../../../../translated_images/tr/LLM-Flow.3285ed8caf4796d7.webp)


### Bir fonksiyon çağrısının öğeleri

#### Kullanıcı Girişi

İlk adım bir kullanıcı mesajı oluşturmaktır. Bu, bir metin girişinin değerini alarak dinamik olarak atanabilir veya burada bir değer atayabilirsiniz. Eğer Chat Completions API ile ilk kez çalışıyorsanız, mesajın `role` ve `content` öğelerini tanımlamamız gerekir.

`role` ya `system` (kuralları oluşturma), `assistant` (model) ya da `user` (son kullanıcı) olabilir. Fonksiyon çağrısı için bunu `user` olarak atayacağız ve bir örnek soru vereceğiz.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Fonksiyon oluşturma.

Sonraki adımda bir fonksiyon ve o fonksiyonun parametrelerini tanımlayacağız. Burada sadece `search_courses` adlı bir fonksiyon kullanacağız ancak birden fazla fonksiyon oluşturabilirsiniz.

**Önemli** : Fonksiyonlar, LLM'ye gönderilen sistem mesajına dahil edilir ve kullanılabilir token miktarınıza dahil edilir.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Tanımlar** 

Fonksiyon tanımı yapısı, her biri kendi özelliklerine sahip birden çok seviyeye sahiptir. İşte iç içe yapının bir dökümü:

**En Üst Düzey Fonksiyon Özellikleri:**

`name` - Çağrılmasını istediğimiz fonksiyonun adı.

`description` - Fonksiyonun nasıl çalıştığına dair açıklama. Burada spesifik ve net olmak önemlidir.

`parameters` - Modelin yanıtında üretmesini istediğiniz değerler ve formatların listesi.

**Parametreler Nesnesi Özellikleri:**

`type` - Parametreler nesnesinin veri tipi (genellikle "object")

`properties` - Modelin yanıtında kullanacağı belirli değerlerin listesi.

**Bireysel Parametre Özellikleri:**

`name` - Özellik anahtarı tarafından örtük olarak tanımlanır (örneğin, "role", "product", "level")

`type` - Bu belirli parametrenin veri tipi (örneğin, "string", "number", "boolean")

`description` - Belirli parametrenin açıklaması.

**İsteğe Bağlı Özellikler:**

`required` - Fonksiyon çağrısının tamamlanması için hangi parametrelerin gerekli olduğunu listeleyen bir dizi.


### Fonksiyon çağrısı yapma  
Bir fonksiyon tanımladıktan sonra, şimdi onu Chat Completion API çağrısına dahil etmemiz gerekiyor. Bunu, isteğe `functions` ekleyerek yapıyoruz. Bu durumda `functions=functions`.  

Ayrıca `function_call` değerini `auto` olarak ayarlama seçeneği de vardır. Bu, fonksiyon çağrısını kendimiz atamak yerine, hangi fonksiyonun çağrılacağına LLM'nin kullanıcı mesajına göre karar vermesine izin vereceğimiz anlamına gelir.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Şimdi yanıtı inceleyelim ve nasıl formatlandığına bakalım:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Fonksiyonun adı çağrılmış ve kullanıcı mesajından, LLM'nin fonksiyonun argümanlarına uyan veriyi bulabildiğini görebilirsiniz.


## 3.Bir Uygulamaya Fonksiyon Çağrılarını Entegre Etme. 


LLM'den biçimlendirilmiş yanıtı test ettikten sonra, şimdi bunu bir uygulamaya entegre edebiliriz. 

### Akışı Yönetme 

Bunu uygulamamıza entegre etmek için, aşağıdaki adımları izleyelim: 

Öncelikle, OpenAI hizmetlerine çağrı yapalım ve mesajı `response_message` adlı bir değişkende saklayalım. 


In [ ]:
response_message = response.choices[0].message

Şimdi Microsoft Learn API'sini çağırarak kurs listesini alacak fonksiyonu tanımlayacağız:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



En iyi uygulama olarak, modelin bir fonksiyonu çağırmak isteyip istemediğine bakacağız. Bundan sonra, mevcut fonksiyonlardan birini oluşturup çağrılan fonksiyonla eşleştireceğiz.  
Ardından, fonksiyonun argümanlarını alıp bunları LLM'den gelen argümanlarla eşleştireceğiz.

Son olarak, fonksiyon çağrısı mesajını ve `search_courses` mesajı tarafından döndürülen değerleri ekleyeceğiz. Bu, LLM'ye kullanıcıya doğal dil kullanarak yanıt vermesi için gereken tüm bilgileri sağlar.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



Şimdi, API JSON formatında bir yanıt yerine doğal dil yanıtı alabilmemiz için güncellenmiş mesajı LLM'ye göndereceğiz.


In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Kod Meydan Okuması

Harika iş! OpenAI Fonksiyon Çağrısı öğreniminize devam etmek için şunları oluşturabilirsiniz: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst  
 - Öğrencilerin daha fazla kurs bulmasına yardımcı olabilecek fonksiyonun daha fazla parametresi. Mevcut API parametrelerini burada bulabilirsiniz:  
 - Öğrencinin ana dili gibi daha fazla bilgi alan başka bir fonksiyon çağrısı oluşturun  
 - Fonksiyon çağrısı ve/veya API çağrısı uygun kurslar döndürmediğinde hata yönetimi oluşturun


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Feragatname**:  
Bu belge, AI çeviri servisi [Co-op Translator](https://github.com/Azure/co-op-translator) kullanılarak çevrilmiştir. Doğruluk için çaba gösterilse de, otomatik çevirilerin hatalar veya yanlışlıklar içerebileceğini lütfen unutmayın. Orijinal belge, kendi dilinde yetkili kaynak olarak kabul edilmelidir. Kritik bilgiler için profesyonel insan çevirisi önerilir. Bu çevirinin kullanımı sonucu oluşabilecek yanlış anlamalar veya yorum hatalarından sorumlu değiliz.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
